In [2]:
from pycaret.classification import *
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
import requests
from bs4 import BeautifulSoup
import statistics
import math

In [3]:
open_file = open('model_columns.pkl', "rb")
model_columns = list(pickle.load(open_file))
open_file.close()

In [4]:
scraped_cols = ['Total shots','Expected goals (xG)','Offsides','Accurate passes percentage','score','Corners','Fouls committed','Shots on target','Blocks']
pred_cols = dict(zip(model_columns, scraped_cols))

In [4]:
pred_cols

{'Shot Attempts': 'Total shots',
 'Total xG': 'Expected goals (xG)',
 'Offsides': 'Offsides',
 'Pass Completion': 'Accurate passes percentage',
 'Goals': 'score',
 'Corners': 'Corners',
 'Fouls Committed': 'Fouls committed',
 'Total Shots on Target': 'Shots on target',
 'Shots Blocked': 'Blocks'}

In [5]:
model = load_model('lr_model')

Transformation Pipeline and Model Successfully Loaded


In [43]:
today = f"{datetime.now():%Y%m%d}"
today = '20220908'
print(today)
game_data = fotmob_game_data(f'https://www.fotmob.com/?date={today}')
all_odds = get_all_odds_links()
days_data = connect_links(game_data, all_odds)

20220908


In [44]:
days_data

{'2022-09-08 13:37:00': [['fc-z%C3%BCrich-vs-arsenal',
   'EUROPA LEAGUE GRP. A',
   'https://www.fotmob.com/match/4010254/matchfacts/fc-z%C3%BCrich-vs-arsenal',
   'https://www.oddschecker.com/football/europa-league/fc-zurich-v-arsenal/winner'],
  ['psv-eindhoven-vs-bod%c3%b8%2fglimt',
   'EUROPA LEAGUE GRP. A',
   'https://www.fotmob.com/match/4010255/matchfacts/psv-eindhoven-vs-bod%c3%b8%2fglimt',
   'https://www.oddschecker.com/football/europa-league/psv-v-bodo-glimt/winner'],
  ['aek-larnaca-vs-rennes',
   'EUROPA LEAGUE GRP. B',
   'https://www.fotmob.com/match/4010293/matchfacts/aek-larnaca-vs-rennes',
   'https://www.oddschecker.com/football/europa-league/aek-larnaca-v-rennes/winner'],
  ['fenerbah%C3%A7e-vs-dynamo-kyiv',
   'EUROPA LEAGUE GRP. B',
   'https://www.fotmob.com/match/4010294/matchfacts/fenerbah%C3%A7e-vs-dynamo-kyiv',
   'https://www.oddschecker.com/football/europa-league/fenerbahce-v-dynamo-kiev/winner']],
 '2022-09-08 13:38:00': [['hjk-vs-real-betis',
   'EUROPA

In [45]:
days_data = {'2022-09-08 13:37:00': [['fc-z%C3%BCrich-vs-arsenal',
   'EUROPA LEAGUE GRP. A',
   'https://www.fotmob.com/match/4010254/matchfacts/fc-z%C3%BCrich-vs-arsenal',
   'https://www.oddschecker.com/football/europa-league/fc-zurich-v-arsenal/winner'],
  ['psv-eindhoven-vs-bod%c3%b8%2fglimt',
   'EUROPA LEAGUE GRP. A',
   'https://www.fotmob.com/match/4010255/matchfacts/psv-eindhoven-vs-bod%c3%b8%2fglimt',
   'https://www.oddschecker.com/football/europa-league/psv-v-bodo-glimt/winner'],
  ['aek-larnaca-vs-rennes',
   'EUROPA LEAGUE GRP. B',
   'https://www.fotmob.com/match/4010293/matchfacts/aek-larnaca-vs-rennes',
   'https://www.oddschecker.com/football/europa-league/aek-larnaca-v-rennes/winner'],
  ['fenerbah%C3%A7e-vs-dynamo-kyiv',
   'EUROPA LEAGUE GRP. B',
   'https://www.fotmob.com/match/4010294/matchfacts/fenerbah%C3%A7e-vs-dynamo-kyiv',
   'https://www.oddschecker.com/football/europa-league/fenerbahce-v-dynamo-kiev/winner']],
 '2022-09-08 13:38:00': [['hjk-vs-real-betis',
   'EUROPA LEAGUE GRP. C',
   'https://www.fotmob.com/match/4010532/matchfacts/hjk-vs-real-betis',
   'https://www.oddschecker.com/football/europa-league/hjk-helsinki-v-real-betis/winner'],
  ['ludogorets-razgrad-vs-roma',
   'EUROPA LEAGUE GRP. C',
   'https://www.fotmob.com/match/4010533/matchfacts/ludogorets-razgrad-vs-roma',
   'https://www.oddschecker.com/football/europa-league/ludogorets-razgrad-v-roma/winner'],
  ['malm%C3%B6-ff-vs-braga',
   'EUROPA LEAGUE GRP. D',
   'https://www.fotmob.com/match/4010661/matchfacts/malm%C3%B6-ff-vs-braga',
   'https://www.oddschecker.com/football/europa-league/malmo-ff-v-braga/winner'],
  ['union-berlin-vs-union-st.gilloise',
   'EUROPA LEAGUE GRP. D',
   'https://www.fotmob.com/match/4010662/matchfacts/union-berlin-vs-union-st.gilloise',
   'https://www.oddschecker.com/football/europa-league/union-berlin-v-union-saint-gilloise/winner']],
 '2022-09-08 13:39:00': [['fiorentina-vs-rfs',
   'EUROPA CONFERENCE LEAGUE GRP. A',
   'https://www.fotmob.com/match/4010194/matchfacts/fiorentina-vs-rfs',
   'https://www.oddschecker.com/football/europa-conference-league/fiorentina-v-rigas-fs/winner'],
  ['hearts-vs-istanbul-ba%C5%9Fak%C5%9Fehir',
   'EUROPA CONFERENCE LEAGUE GRP. A',
   'https://www.fotmob.com/match/4010193/matchfacts/hearts-vs-istanbul-ba%C5%9Fak%C5%9Fehir',
   'https://www.oddschecker.com/football/europa-conference-league/hearts-v-istanbul-basaksehir/winner'],
  ['anderlecht-vs-silkeborg',
   'EUROPA CONFERENCE LEAGUE GRP. B',
   'https://www.fotmob.com/match/4010217/matchfacts/anderlecht-vs-silkeborg',
   'https://www.oddschecker.com/football/europa-conference-league/anderlecht-v-silkeborg-if/winner'],
  ['austria-wien-vs-hapoel-beer-sheva',
   'EUROPA CONFERENCE LEAGUE GRP. C',
   'https://www.fotmob.com/match/4010243/matchfacts/austria-wien-vs-hapoel-beer-sheva',
   'https://www.oddschecker.com/football/europa-conference-league/fk-austria-vienna-v-hapoel-beer-sheva/winner']],
 '2022-09-08 13:40:00': [['villarreal-vs-lech-poznan',
   'EUROPA CONFERENCE LEAGUE GRP. C',
   'https://www.fotmob.com/match/4010242/matchfacts/villarreal-vs-lech-poznan',
   'https://www.oddschecker.com/football/europa-conference-league/villarreal-v-lech-poznan/winner'],
  ['nice-vs-1.-fc-k%C3%B6ln',
   'EUROPA CONFERENCE LEAGUE GRP. D',
   'https://www.fotmob.com/match/4010267/matchfacts/nice-vs-1.-fc-k%C3%B6ln',
   'https://www.oddschecker.com/football/europa-conference-league/nice-v-cologne/winner'],
  ['slovacko-vs-partizan-beograd',
   'EUROPA CONFERENCE LEAGUE GRP. D',
   'https://www.fotmob.com/match/4010268/matchfacts/slovacko-vs-partizan-beograd',
   'https://www.oddschecker.com/football/europa-conference-league/slovacko-v-partizan-belgrade/winner'],
  ['ballkani-vs-cfr-cluj',
   'EUROPA CONFERENCE LEAGUE GRP. G',
   'https://www.fotmob.com/match/4010580/matchfacts/ballkani-vs-cfr-cluj',
   'https://www.oddschecker.com/football/europa-conference-league/kf-ballkani-v-cfr-cluj/winner']],
 '2022-09-08 15:52:00': [['manchester-united-vs-real-sociedad',
   'EUROPA LEAGUE GRP. E',
   'https://www.fotmob.com/match/4010305/matchfacts/manchester-united-vs-real-sociedad',
   'https://www.oddschecker.com/football/europa-league/man-utd-v-real-sociedad/winner'],
  ['omonia-nicosia-vs-fc-sheriff',
   'EUROPA LEAGUE GRP. E',
   'https://www.fotmob.com/match/4010306/matchfacts/omonia-nicosia-vs-fc-sheriff',
   'https://www.oddschecker.com/football/europa-league/omonia-nicosia-v-sheriff-tiraspol/winner'],
  ['lazio-vs-feyenoord',
   'EUROPA LEAGUE GRP. F',
   'https://www.fotmob.com/match/4010375/matchfacts/lazio-vs-feyenoord',
   'https://www.oddschecker.com/football/europa-league/lazio-v-feyenoord/winner'],
  ['sturm-graz-vs-midtjylland',
   'EUROPA LEAGUE GRP. F',
   'https://www.fotmob.com/match/4010374/matchfacts/sturm-graz-vs-midtjylland',
   'https://www.oddschecker.com/football/europa-league/sk-sturm-graz-v-midtjylland/winner']],
 '2022-09-08 15:53:00': [['sc-freiburg-vs-qarabag-fk',
   'EUROPA LEAGUE GRP. G',
   'https://www.fotmob.com/match/4010544/matchfacts/sc-freiburg-vs-qarabag-fk',
   'https://www.oddschecker.com/football/europa-league/sc-freiburg-v-fk-qarabag/winner'],
  ['nantes-vs-olympiacos',
   'EUROPA LEAGUE GRP. G',
   'https://www.fotmob.com/match/4010545/matchfacts/nantes-vs-olympiacos',
   'https://www.oddschecker.com/football/europa-league/nantes-v-olympiakos/winner'],
  ['ferencvaros-vs-trabzonspor',
   'EUROPA LEAGUE GRP. H',
   'https://www.fotmob.com/match/4010650/matchfacts/ferencvaros-vs-trabzonspor',
   'https://www.oddschecker.com/football/europa-league/ferencvarosi-tc-v-trabzonspor/winner'],
  ['fk-crvena-zvezda-vs-monaco',
   'EUROPA LEAGUE GRP. H',
   'https://www.fotmob.com/match/4010649/matchfacts/fk-crvena-zvezda-vs-monaco',
   'https://www.oddschecker.com/football/europa-league/crvena-zvezda-v-monaco/winner']],
 '2022-09-08 15:54:00': [['west-ham-united-vs-fc-fcsb',
   'EUROPA CONFERENCE LEAGUE GRP. B',
   'https://www.fotmob.com/match/4010218/matchfacts/west-ham-united-vs-fc-fcsb',
   'https://www.oddschecker.com/football/europa-conference-league/west-ham-v-fcsb/winner'],
  ['fc-vaduz-vs-apollon-limassol',
   'EUROPA CONFERENCE LEAGUE GRP. E',
   'https://www.fotmob.com/match/4010319/matchfacts/fc-vaduz-vs-apollon-limassol',
   'https://www.oddschecker.com/football/europa-conference-league/vaduz-v-apollon-limassol/winner'],
  ['sc-dnipro-1-vs-az-alkmaar',
   'EUROPA CONFERENCE LEAGUE GRP. E',
   'https://www.fotmob.com/match/4010320/matchfacts/sc-dnipro-1-vs-az-alkmaar',
   'https://www.oddschecker.com/football/europa-conference-league/dnipro-1-v-az/winner'],
  ['molde-vs-gent',
   'EUROPA CONFERENCE LEAGUE GRP. F',
   'https://www.fotmob.com/match/4010386/matchfacts/molde-vs-gent',
   'https://www.oddschecker.com/football/europa-conference-league/molde-v-gent/winner']],
 '2022-09-08 15:55:00': [['shamrock-rovers-vs-djurg%C3%A5rden',
   'EUROPA CONFERENCE LEAGUE GRP. F',
   'https://www.fotmob.com/match/4010387/matchfacts/shamrock-rovers-vs-djurg%C3%A5rden',
   'https://www.oddschecker.com/football/europa-conference-league/shamrock-rovers-v-djurgarden/winner'],
  ['sivasspor-vs-slavia-prague',
   'EUROPA CONFERENCE LEAGUE GRP. G',
   'https://www.fotmob.com/match/4010581/matchfacts/sivasspor-vs-slavia-prague',
   'https://www.oddschecker.com/football/europa-conference-league/sivasspor-v-slavia-prague/winner'],
  ['basel-vs-pyunik',
   'EUROPA CONFERENCE LEAGUE GRP. H',
   'https://www.fotmob.com/match/4010674/matchfacts/basel-vs-pyunik',
   'https://www.oddschecker.com/football/europa-conference-league/basel-v-pyunik-yerevan/winner'],
  ['slovan-bratislava-vs-zalgiris-vilnius',
   'EUROPA CONFERENCE LEAGUE GRP. H',
   'https://www.fotmob.com/match/4010675/matchfacts/slovan-bratislava-vs-zalgiris-vilnius',
   'https://www.oddschecker.com/football/europa-conference-league/slovan-bratislava-v-fk-zalgiris-vilnius/winner']]}

In [51]:
c = days_data.copy()
todays_data = []

while True:
    for clock in c.copy().keys():
        if datetime.strptime(clock, "%Y-%m-%d %H:%M:%S") <= datetime.now():
            stats_urls = [x[2] for x in c[clock]]
            odds_urls = [x[3] for x in c[clock]]
            print(stats_urls, odds_urls)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                r = executor.map(stats_odds, stats_urls, odds_urls)
            for result in r:
                todays_data.append(result)
            del c[clock]
    print(f'{len(c)} halftimes left -- {len(todays_data)}\n')
    if len(c) == 0:
        print('all done')
        break
    time.sleep(60)

open_file = open(f'{today}_data.pkl', "wb")
pickle.dump(todays_data, open_file)
open_file.close()

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

8 halftimes left -- 0

['https://www.fotmob.com/match/4010254/matchfacts/fc-z%C3%BCrich-vs-arsenal', 'https://www.fotmob.com/match/4010255/matchfacts/psv-eindhoven-vs-bod%c3%b8%2fglimt', 'https://www.fotmob.com/match/4010293/matchfacts/aek-l

['Draw', 'Slovacko', 'Partizan Belgrade']
[36.0, 1.0066666666666666, 95.0]
slovacko-vs-partizan-beograd
Predicted winner is 1

draw: predicted prob: 3.2% - oddschecker: 2.78% (36.0) ---- bet 0.00215
home: predicted prob: 96.7% - oddschecker: 99.3% (1.007) ---- bet no value
away: predicted prob: 0.1% - oddschecker: 1.05% (95.0) ---- bet no value


['Draw', 'KF Ballkani', 'CFR Cluj']
[2.15, 4.0, 2.625]
ballkani-vs-cfr-cluj
Predicted winner is 1

draw: predicted prob: 21.6% - oddschecker: 46.51% (2.15) ---- bet no value
home: predicted prob: 52.900000000000006% - oddschecker: 25.0% (4.0) ---- bet 0.186
away: predicted prob: 25.5% - oddschecker: 38.1% (2.625) ---- bet no value


list index out of range
['Draw', 'Nice', 'Cologne']
4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16

4 halftimes left -- 16



['Draw', 'West Ham', 'FCSB']
[2.947368421052632, 2.35, 3.2]
west-ham-united-vs-fc-fcsb
Predicted winner is 2

draw: predicted prob: 36.5% - oddschecker: 33.93% (2.947) ---- bet 0.01945
home: predicted prob: 12.4% - oddschecker: 42.55% (2.35) ---- bet no value
away: predicted prob: 51.1% - oddschecker: 31.25% (3.2) ---- bet 0.14435


['Draw', 'Dnipro-1', 'AZ']
[2.25, 6.0, 2.1]
sc-dnipro-1-vs-az-alkmaar
Predicted winner is 2

draw: predicted prob: 29.4% - oddschecker: 44.44% (2.25) ---- bet no value
home: predicted prob: 31.3% - oddschecker: 16.67% (6.0) ---- bet 0.0878
away: predicted prob: 39.4% - oddschecker: 47.62% (2.1) ---- bet no value


['Draw', 'Vaduz', 'Apollon Limassol']
[2.2, 3.75, 2.7]
fc-vaduz-vs-apollon-limassol
Predicted winner is 2

draw: predicted prob: 29.7% - oddschecker: 45.45% (2.2) ---- bet no value
home: predicted prob: 33.300000000000004% - oddschecker: 26.67% (3.75) ---- bet 0.04525
away: predicted prob: 37.0% - oddschecker: 37.04% (2.7) ---- bet no value


['Dr

In [27]:
todays_data

[{'Expected goals (xG)': ['0.35', '0.07'],
  'Total shots': ['4', '3'],
  'Big chances': ['1', '0'],
  'Big chances missed': ['0', '0'],
  'Accurate passes': ['152', '305'],
  'Accurate passes percentage': [0.81, 0.84],
  'Fouls committed': ['6', '8'],
  'Offsides': ['0', '3'],
  'Corners': ['1', '3'],
  'Shots off target': ['1', '0'],
  'Shots on target': ['2', '1'],
  'Blocked shots': ['1', '2'],
  'Hit woodwork': ['0', '0'],
  'Shots inside box': ['2', '1'],
  'Shots outside box': ['2', '2'],
  'xG open play': ['0.31', '0.07'],
  'xG set play': ['0.04', '0.01'],
  'xG on target (xGOT)': ['1.15', '0.06'],
  'Passes': ['188', '362'],
  'Own half': ['76', '72'],
  'Opposition half': ['76', '233'],
  'Accurate long balls': ['10', '11'],
  'Accurate long balls percentage': [0.42, 0.5],
  'Accurate crosses': ['0', '2'],
  'Accurate crosses percentage': [0.0, 0.22],
  'Throws': ['9', '3'],
  'Yellow cards': ['0', '0'],
  'Red cards': ['0', '0'],
  'Tackles won': ['3', '7'],
  'Tackles won 

In [278]:
open_file = open('20220828_data.pkl', "rb")
xcxc = pickle.load(open_file)
open_file.close()

In [7]:
def predicted_probs(results):
    d = {}
    for k, v in results.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    for_pred = [[d[key] for key in pred_cols.values()]]
    pred = model[-1].predict_proba(for_pred)
    return pred

In [8]:
def make_bet(decimal_odds, predicted_prob, k_weight):
    if 1/decimal_odds >= predicted_prob:
        return 'no value'
    else:
        return kelly(predicted_prob, decimal_odds, k_weight)

In [9]:
def print_advice(results):
    try:
        d_h_a = ['draw'] + results['odds_url'].split('/')[-2].split('-v-')
        order_decimal_odds = [getname(odds_name, list(results['odds'].keys())) for odds_name in d_h_a]
        print(order_decimal_odds)
        p_probs = predicted_probs(results)[0]
        predicted = np.argmax(p_probs)
        decimal_odds = [results['odds'][key] for key in order_decimal_odds]
        print(decimal_odds)
        bet_made = {}
        print('\033[1m' + results['match_name'] + '\033[0m')
        print(f'Predicted winner is {predicted}\n')
        for p_prob, d_odd, win in zip(p_probs, decimal_odds, ['draw', 'home','away']):
            p_prob = round(p_prob, 3)
            d_odd = round(d_odd, 3)
            size = make_bet(d_odd, p_prob, 0.5)
            print(f'{win}: predicted prob: {p_prob*100}% - oddschecker: {round(1/d_odd*100,2)}% ({d_odd}) ---- bet {size}')
            bet_made[win] = size
        print('\n')
        bet_made['odds_checker_sum'] = np.sum([1/o for o in decimal_odds])
        bet_made['winner'] = predicted
        bet_made['predicted_probs'] = np.round(p_probs, 3)
        bets = {'bet':bet_made} 
        row = {**results, **bets}
    except:
        row = results
    return row

In [10]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [11]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [37]:
def get_stats(url):
    try:
        page = requests.get(url)
        time.sleep(random.randint(0, 4))
        soup = BeautifulSoup(page.content, 'html.parser')
        ht = soup.find("span", {"class": "css-lv1jm0-bottomRow"}).text
        if ht != 'Half-Time':
                print('not halftime just yet ... sleeping\n')
                time.sleep(3*60)
                ht = soup.find("span", {"class": "css-lv1jm0-bottomRow"}).text
        div1 = soup.find_all("li", {"class": "css-radwzz-Stat e683amr6"})
        all_stats = {k[1]: [k[0], k[2]] for k in [z.split('|') for z in [x for x in [d.get_text('|') for d in div1]
                  if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
        score = [score[0], score[1]]
        posession = soup.find("div", {"class": "css-7s52se-PossessionWheel e683amr3"}).get_text('|').split('|')
        posession = [posession[1], posession[3]]
        all_stats['posession'] = posession
        all_stats['score'] = score
        game_stats = {}
        for name, stats in all_stats.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
        match_name = url.split('/')[-1]
        game_stats['match_name'] = match_name
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except Exception as e:
        print(e)
        game_stats = {}
    return game_stats

In [36]:
get_stats('https://www.fotmob.com/match/3900365/matchfacts/az-alkmaar-vs-nec-nijmegen')

not halftime just yet ... sleeping



{'Expected goals (xG)': ['2.62', '0.07'],
 'Total shots': ['30', '4'],
 'Big chances': ['4', '0'],
 'Big chances missed': ['4', '0'],
 'Accurate passes': ['534', '266'],
 'Accurate passes percentage': [0.89, 0.76],
 'Fouls committed': ['7', '10'],
 'Offsides': ['1', '0'],
 'Corners': ['8', '3'],
 'Shots off target': ['8', '2'],
 'Shots on target': ['13', '2'],
 'Blocked shots': ['9', '0'],
 'Hit woodwork': ['0', '0'],
 'Shots inside box': ['17', '1'],
 'Shots outside box': ['13', '3'],
 'xG first half': ['0.37', '0.04'],
 'xG second half': ['2.25', '0.03'],
 'xG open play': ['2.35', '0.04'],
 'xG set play': ['0.27', '0.03'],
 'xG on target (xGOT)': ['1.74', '0.89'],
 'Passes': ['600', '349'],
 'Own half': ['197', '199'],
 'Opposition half': ['337', '67'],
 'Accurate long balls': ['29', '23'],
 'Accurate long balls percentage': [0.59, 0.4],
 'Accurate crosses': ['6', '3'],
 'Accurate crosses percentage': [0.33, 0.33],
 'Throws': ['18', '13'],
 'Yellow cards': ['1', '2'],
 'Red cards': [

In [42]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership',
             'ITALY - SERIE A':'italy/serie-a', 'CHAMPIONS LEAGUE GRP. A': 'champions-league',
             'CHAMPIONS LEAGUE GRP. B': 'champions-league', 'CHAMPIONS LEAGUE GRP. C': 'champions-league',
             'CHAMPIONS LEAGUE GRP. D': 'champions-league','CHAMPIONS LEAGUE GRP. E': 'champions-league',
             'CHAMPIONS LEAGUE GRP. F': 'champions-league','CHAMPIONS LEAGUE GRP. G': 'champions-league',
             'CHAMPIONS LEAGUE GRP. H': 'champions-league', 'EUROPA LEAGUE GRP. A': 'europa-league',
             'EUROPA LEAGUE GRP. B': 'europa-league', 'EUROPA LEAGUE GRP. C': 'europa-league',
             'EUROPA LEAGUE GRP. D': 'europa-league', 'EUROPA LEAGUE GRP. E': 'europa-league',
             'EUROPA LEAGUE GRP. F': 'europa-league', 'EUROPA LEAGUE GRP. G': 'europa-league',
             'EUROPA LEAGUE GRP. H': 'europa-league', 'EUROPA LEAGUE GRP. A': 'europa-league',
             'EUROPA CONFERENCE LEAGUE GRP. A': 'uefa-europa-conference-league',
             'EUROPA CONFERENCE LEAGUE GRP. B': 'uefa-europa-conference-league',
             'EUROPA CONFERENCE LEAGUE GRP. C': 'uefa-europa-conference-league',
             'EUROPA CONFERENCE LEAGUE GRP. D': 'uefa-europa-conference-league',
             'EUROPA CONFERENCE LEAGUE GRP. E': 'uefa-europa-conference-league',
             'EUROPA CONFERENCE LEAGUE GRP. F': 'uefa-europa-conference-league',
             'EUROPA CONFERENCE LEAGUE GRP. G': 'uefa-europa-conference-league',
             'EUROPA CONFERENCE LEAGUE GRP. H': 'uefa-europa-conference-league'}

def fotmob_game_data(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-1f0alcw-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_leagues.keys():
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [16]:
def get_all_odds_links():
    odds_links = {}
    for f_league, url_league in xg_leagues.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = 'https://www.oddschecker.com/football/' + url_league
        driver.get(url)
        driver.implicitly_wait(3)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [17]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    r = {}
    test = scraper_data.copy()
    for date, data in test.copy().items():
        if len(data) > 5:
            x = [data[i:i + 4] for i in range(0, len(data), 4)]
            d = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
            groups = len(x)
            times = [str(d + timedelta(minutes = i * 1)) for i in range(-groups//2+1,groups//2+1, 1)]
            new = dict(zip(times, x))
            del test[date]
            r = {**r, **new}
        else:
            r[date] = data
    scraper_data = dict(sorted(r.items(), key=lambda x: x[0]))
    return scraper_data

In [28]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {'odds_url':url, 'odds': '', 'full_odds': ''}
        driver.implicitly_wait(1)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, features="html.parser")
        odds_containers = soup.find_all("div", {"class": "oddsAreaWrapper_o17xb9rs RowLayout_refg9ta"})
        names = soup.find_all("div", {"class": "BetRowLeftBetContent_b2f00kt"})
        names = [n.text for n in names]
        odds_containers = [container.find_all('button') for container in odds_containers]
        odds = [[z.text for z in container] for container in odds_containers]
        odds = [[float(Fraction(j.replace(' ',''))) + 1.0 for j in o][:-3] for o in odds]
        median_odds = [statistics.median(o) for o in odds]
        median_odds = dict(zip(names, median_odds))
        odds = {'odds_url':url, 'odds': median_odds, 'full_odds': odds}
    except Exception as e:
        print(e)
        return {'odds_url':url, 'odds': '', 'full_odds': ''}
    driver.quit()
    return odds

In [50]:
get_odds('https://www.oddschecker.com/football/europa-conference-league/anderlecht-v-silkeborg-if/winner')

{'odds_url': 'https://www.oddschecker.com/football/europa-conference-league/anderlecht-v-silkeborg-if/winner',
 'odds': {'Anderlecht': 1.7272727272727273, 'Draw': 3.3, 'Silkeborg IF': 5.0},
 'full_odds': [[1.7272727272727273,
   1.7272727272727273,
   1.6666666666666665,
   1.65,
   1.7272727272727273,
   1.6666666666666665,
   1.6666666666666665,
   1.75,
   1.6666666666666665,
   1.75,
   1.7272727272727273,
   1.7857142857142856,
   1.7,
   1.7647058823529411,
   1.8,
   1.7,
   1.6666666666666665,
   1.6666666666666665,
   1.75,
   1.7,
   1.75,
   1.6666666666666665,
   1.8333333333333335],
  [3.4,
   3.25,
   3.4,
   3.4,
   3.0,
   3.4,
   3.3,
   3.3,
   3.5,
   3.3,
   3.25,
   3.5454545454545454,
   3.1,
   3.1538461538461537,
   3.2,
   3.4,
   3.5,
   3.1,
   3.5,
   3.3,
   3.25,
   3.3,
   3.6470588235294117],
  [5.0,
   5.0,
   5.5,
   4.8,
   4.5,
   5.5,
   4.8,
   5.0,
   5.0,
   4.75,
   5.0,
   5.7,
   4.75,
   4.75,
   4.846153846153847,
   5.0,
   5.0,
   4.6,
   

In [19]:
def stats_odds(stats_url, odds_url):
    stats = get_stats(stats_url)
    time.sleep(20)
    odds = get_odds(odds_url)
    row = {**stats, **odds}
    result = print_advice(row)
    return result